In [0]:
# ensemble은 여러개의 cnn model을 만들어서 이용 
# 객체를 만들어서 각 개체에 CNN model을 하나씩 저장 


In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import numpy as np

# 그래프 초기화
tf.reset_default_graph()

class CnnModel:
    # 객체 생성
    def __init__(self):
        print("객체생성") 
        self.build_graph() # self를 안붙이면 변수의 가동 범위가 해당 함수밖에 안됨 (self붙이면 class 밖에서도 사용가능)
        self.get_accuracy()
    
    # 그래프 그리는 함수
    def build_graph(self):
        print("텐서플로우 그래프를 그린다")

    # 그래프 그린 후 학습
    def train_graph(self):
        print("그래프 그린 후 학습시킨다")

    # 정확도 구하기
    def get_accuracy(self):
        print("정확도를 구한다")

    # 예측
    def get_prediction(self):
        print("예측값을 구한다")


# 모델 10개 만들기 -> 객체 10개 만들기
num_of_model = 3
models = [CnnModel() for x in range(num_of_model)]



객체생성
텐서플로우 그래프를 그린다
정확도를 구한다
객체생성
텐서플로우 그래프를 그린다
정확도를 구한다
객체생성
텐서플로우 그래프를 그린다
정확도를 구한다


In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import numpy as np

# 그래프 초기화
tf.reset_default_graph()

class CnnModel:
    # 객체 생성
    def __init__(self,session,data):
        print("객체생성")
        self.build_graph()
        self.get_accuracy()
        self.mnist = data

        # Session & 초기화
        self.sess = session 
        # self train에서 줘도 되지만, 그러면 각 그래프 그릴때마다 생성하므로.. 1개만 만들면 되니까  여기서 만든다. 
        # 이렇게 session 객체 하나만 만들고 나서 뒤에 하나씩 넘겨주면 모든 그래프들이 session을 이용할 수 있다. 
        # 굳이 여러개 만들 필요 없다.
        
    
    # 그래프 그리는 함수
    def build_graph(self):
        print("텐서플로우 그래프를 그린다")

        
        # 2.1 placeholder
        self.X = tf.placeholder(shape=[None,784], dtype=tf.float32)  # 학습할 때 필요하니까 다 self
        self.Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
        self.dropout_rate = tf.placeholder(dtype=tf.float32)  

        # 2.2 Convolution Layer
        x_img = tf.reshape(self.X,[-1,28,28,1])  # self 붙일 필요 없음. 다른 함수에서는 이거 안쓰므로

        L1 = tf.layers.conv2d(inputs=x_img, filters=32, kernel_size=[3,3], padding="SAME",
                            strides=1,activation=tf.nn.relu)  
        L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2],padding="SAME", strides=2)
        L1 = tf.layers.dropout(inputs=L1, rate=self.dropout_rate)

        L2 = tf.layers.conv2d(inputs=L1, filters=64, kernel_size=[3,3], padding="SAME",
                            strides=1, activation=tf.nn.relu) 
        L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2],padding="SAME", strides=2)
        L2 = tf.layers.dropout(inputs=L2, rate=self.dropout_rate)

        # 2.3 FC layer -> dense layer 
        L2 = tf.reshape(L2, [-1,7*7*64])
        dense1 = tf.layers.dense(inputs=L2, units=256, activation=tf.nn.relu)
        dense1 = tf.nn.dropout(dense1, rate=self.dropout_rate)  

        dense2 = tf.layers.dense(inputs=dense1, units=128, activation=tf.nn.relu)
        dense2 = tf.nn.dropout(dense2, rate=self.dropout_rate)

        dense3 = tf.layers.dense(inputs=dense2, units=512, activation=tf.nn.relu)
        dense3 = tf.nn.dropout(dense3, rate=self.dropout_rate)

        self.H = tf.layers.dense(inputs=dense3, units=10)   # 외부에서 prediction할 때 쓸거니까 self

        # cost 
        self.cost = tf.losses.softmax_cross_entropy(self.Y,self.H)

        # train
        self.train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(self.cost)

        
    # 그래프 그린 후 학습
    def train_graph(self):  # self를 안붙이면 변수의 가동 범위가 해당 함수밖에 안됨
        print("그래프 그린 후 학습시킨다")

        sess.run(tf.global_variables_initializer()) # session 초기화
        # 학습 - epoch, batch 등 
        num_of_epoch = 30
        batch_size = 100
        for step in range(num_of_epoch):
            num_of_iter = int(self.mnist.train.num_examples/batch_size)
            cost_val = 0
            for i in range(num_of_iter):
                batch_x, batch_y = self.mnist.train.next_batch(batch_size)        
                _, cost_val = sess.run([self.train,self.cost],feed_dict={self.X:batch_x,
                                                                         self.Y:batch_y,
                                                                         self.dropout_rate:0.3})
            if step%3==0:
                print("cost:{}".format(cost_val))


    # 정확도 구하기
    def get_accuracy(self):
        print("정확도를 구한다")

    # 예측
    def get_prediction(self):
        print("예측값을 구한다")


# 모델 10개 만들기 -> 객체 10개 만들기
sess = tf.Session()


num_of_model = 3
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)
models = [CnnModel(sess,mnist) for x in range(num_of_model)]
for i in range(num_of_model):
    models[i].train_graph()


Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
객체생성
텐서플로우 그래프를 그린다
정확도를 구한다
객체생성
텐서플로우 그래프를 그린다
정확도를 구한다
객체생성
텐서플로우 그래프를 그린다
정확도를 구한다
그래프 그린 후 학습시킨다
cost:0.07084350287914276
cost:0.050674811005592346
cost:0.00029668299248442054
cost:0.05808315426111221
cost:0.0009112574625760317
cost:0.014048843644559383
cost:0.019814178347587585
cost:0.002343993866816163
cost:0.0003486480563879013
cost:0.035478122532367706
그래프 그린 후 학습시킨다
cost:0.09557727724313736
cost:0.04002121835947037
cost:0.031445685774087906
cost:0.0657741129398346
cost:0.05106642097234726
cost:0.017868446186184883
cost:0.0009873938979580998
cost:9.372635395266116e-05
cost:0.011062707751989365
cost:0.0001176832229248248
그래프 그린 후 학습시킨다
cost:0.0923551619052887
cost:0.004586231894791126
cost:0.009499821811914444
cost:0.02640586346387863
cost:0.003155344631522894
cost:0.005182958208

In [17]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import numpy as np

# 그래프 초기화
tf.reset_default_graph()

class CnnModel:
    # 객체 생성
    def __init__(self,session,data):
        print("객체생성")
        self.build_graph()
        #self.get_accuracy()
        self.mnist = data
        self.train_graph()

        # Session & 초기화
        self.sess = session 
        # self train에서 줘도 되지만, 그러면 각 그래프 그릴때마다 생성하므로.. 1개만 만들면 되니까  여기서 만든다. 
        # 이렇게 session 객체 하나만 만들고 나서 뒤에 하나씩 넘겨주면 모든 그래프들이 session을 이용할 수 있다. 
        # 굳이 여러개 만들 필요 없다.
        
    
    # 그래프 그리는 함수
    def build_graph(self):
        print("텐서플로우 그래프를 그린다")

        
        # 2.1 placeholder
        self.X = tf.placeholder(shape=[None,784], dtype=tf.float32)  # 학습할 때 필요하니까 다 self
        self.Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
        self.dropout_rate = tf.placeholder(dtype=tf.float32)  

        # 2.2 Convolution Layer
        x_img = tf.reshape(self.X,[-1,28,28,1])  # self 붙일 필요 없음. 다른 함수에서는 이거 안쓰므로

        L1 = tf.layers.conv2d(inputs=x_img, filters=32, kernel_size=[3,3], padding="SAME",
                            strides=1,activation=tf.nn.relu)  
        L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2],padding="SAME", strides=2)
        L1 = tf.layers.dropout(inputs=L1, rate=self.dropout_rate)

        L2 = tf.layers.conv2d(inputs=L1, filters=64, kernel_size=[3,3], padding="SAME",
                            strides=1, activation=tf.nn.relu) 
        L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2],padding="SAME", strides=2)
        L2 = tf.layers.dropout(inputs=L2, rate=self.dropout_rate)

        # 2.3 FC layer -> dense layer 
        L2 = tf.reshape(L2, [-1,7*7*64])
        dense1 = tf.layers.dense(inputs=L2, units=256, activation=tf.nn.relu)
        dense1 = tf.nn.dropout(dense1, rate=self.dropout_rate)  

        dense2 = tf.layers.dense(inputs=dense1, units=128, activation=tf.nn.relu)
        dense2 = tf.nn.dropout(dense2, rate=self.dropout_rate)

        dense3 = tf.layers.dense(inputs=dense2, units=512, activation=tf.nn.relu)
        dense3 = tf.nn.dropout(dense3, rate=self.dropout_rate)

        self.H = tf.layers.dense(inputs=dense3, units=10)   # 외부에서 prediction할 때 쓸거니까 self

        # cost 
        self.cost = tf.losses.softmax_cross_entropy(self.Y,self.H)

        # train
        self.train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(self.cost)

        
    # 그래프 그린 후 학습
    def train_graph(self):  # self를 안붙이면 변수의 가동 범위가 해당 함수밖에 안됨
        print("그래프 그린 후 학습시킨다")

        sess.run(tf.global_variables_initializer()) # session 초기화
        # 학습 - epoch, batch 등 
        num_of_epoch = 10
        batch_size = 100
        for step in range(num_of_epoch):
            num_of_iter = int(self.mnist.train.num_examples/batch_size)
            cost_val = 0
            for i in range(num_of_iter):
                batch_x, batch_y = self.mnist.train.next_batch(batch_size)        
                _, cost_val = sess.run([self.train,self.cost],feed_dict={self.X:batch_x,
                                                                         self.Y:batch_y,
                                                                         self.dropout_rate:0.3})
            # if step%3==0:
            #     print("cost:{}".format(cost_val))
        #return self.H_val
    
    def get_h(self):
        self.H_val = sess.run(self.H, feed_dict={self.X: self.mnist.test.images,
                                                 self.dropout_rate:0})
        #print(self.H_val)
        return self.H_val
        

    def get_accuracy(self):
        print("정확도를 구한다")


    def get_prediction(self):
        print("예측값을 구한다")

# 모델 10개 만들기 -> 객체 10개 만들기
sess = tf.Session()


num_of_model = 3
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)
models = [CnnModel(sess,mnist) for x in range(num_of_model)]

model_np = np.zeros([mnist.test.num_examples,10])
for i in range(num_of_model):
    model_np = model_np + models[i].get_h()

predict_result = np.argmax(model_np,1)
true_result = np.argmax(mnist.test.labels,1)
correct = np.equal(predict_result,true_result)
accuracy = np.mean(correct)
print(accuracy)


Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
객체생성
텐서플로우 그래프를 그린다
그래프 그린 후 학습시킨다
객체생성
텐서플로우 그래프를 그린다
그래프 그린 후 학습시킨다
객체생성
텐서플로우 그래프를 그린다
그래프 그린 후 학습시킨다
0.9921


In [0]:
# 정확도 구하기
def get_accuracy(self):
    print("정확도를 구한다")

    predict = tf.argmax(self.H,1)
    correct = tf.equal(predict, tf.argmax(self.Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

    print("정확도:{}".format(sess.run(accuracy, feed_dict={self.X:mnist.test.images,
                                                            self.Y:mnist.test.labels,
                                                            self.dropout_rate:0})))
# 예측
def get_prediction(self):
    print("예측값을 구한다")
    self.result = sess.run(self.H, feed_dict={self.X:test_df})

    predict = tf.cast(self.result > 0.5, dtype=tf.int32)
    self.result = sess.run(predict)
    print(self.result)